In [1]:
import sys
import subprocess
import re
import os
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cmath
import operator
import random
from operator import itemgetter, attrgetter
import matplotlib.cm as cm
import vtk

## import MI measures
from mi_functions import *

In [2]:
totPercentage = 1

In [3]:
# isabel
file1 = '../Data/Isabel_vti/isabel_p_25.vti'
file2 = '../Data/Isabel_vti/isabel_vel_25.vti'
arrayName1 = 'Pressure'
arrayName2 = 'Velocity'
data_set = 'isabel'
samp_type = 'pmi' # random, pmi


# # ## Asteroid
# file1 = '../Data/Asteroid/tev.vti'
# file2 = '../Data/Asteroid/v02.vti'
# arrayName1 = 'tev'
# arrayName2 = 'v02'
# outfile = '../output/joint_pmi_sampled_asteroid_' + arrayName1 + '_' + arrayName2 +  '.vtp'
# pmivol_file = '../output/pmi_vol_tev_v02.vti'

# ## Nyx: high res
# file1 = '/Users/sdutta/Data/Nyx/Density_small.vti'
# file2 = '/Users/sdutta/Data/Nyx/Temperature_small.vti'
# arrayName1 = 'Density'
# arrayName2 = 'Temperature'
# outfile = '../output/joint_pmi_sampled_nyx.vtp'
# pmivol_file = '../output/pmi_vol_dens_temp_nyx.vti'

# ## Nyx
# file1 = '../Data/Nyx/density_1.vti'
# file2 = '../Data/Nyx/temperature_1.vti'
# arrayName1 = 'logField'
# arrayName2 = 'logField'
# outfile = '../output/joint_pmi_sampled_nyx.vtp'
# pmivol_file = '../output/pmi_vol_dens_temp_nyx.vti'

# ## Combustion
# file1 = '../Data/Combustion/combustion_mixfrac.vti'
# file2 = '../Data/Combustion/combustion_chi.vti'
# arrayName1 = 'mixfrac'
# arrayName2 = 'chi'
# outfile = '../output/joint_pmi_sampled_combustion.vtp'
# pmivol_file = '../output/pmi_vol_mixfrac_chi.vti'

# ## Mfix
# file1 = '../Data/Mfix/original_density_out.vti'
# file2 = '../Data/Mfix/gradient_out.vti'
# arrayName1 = 'ImageScalars'
# arrayName2 = 'ImageScalars'
# outfile = '../output/joint_pmi_sampled_mfix.vtp'
# pmivol_file = '../output/pmi_vol_gradient_density.vti'

In [4]:
# ## load vti data
R1 = vtk.vtkXMLImageDataReader()
R1.SetFileName(file1)
R1.Update()
dataArray1 = R1.GetOutput().GetPointData().GetArray(arrayName1)

R2 = vtk.vtkXMLImageDataReader()
R2.SetFileName(file2)
R2.Update()
dataArray2 = R2.GetOutput().GetPointData().GetArray(arrayName2)

var1=np.zeros(dataArray1.GetNumberOfTuples()) 
var2=np.zeros(dataArray2.GetNumberOfTuples()) 

for i in range(dataArray1.GetNumberOfTuples()):
    var1[i] = dataArray1.GetTuple1(i)
    var2[i] = dataArray2.GetTuple1(i)

dims = R1.GetOutput().GetDimensions()

min_var1 = np.min(var1)
max_var1 = np.max(var1)

min_var2 = np.min(var2)
max_var2 = np.max(var2)

print R1.GetOutput().GetSpacing()
print R1.GetOutput().GetOrigin()

(1.0, 1.0, 1.0)
(0.0, 0.0, 0.0)


In [5]:
if samp_type == 'pmi':
    mixinig_factor_PMI = 1
elif samp_type == 'random':
    mixinig_factor_PMI = 0
    
percentageToStroe = totPercentage*mixinig_factor_PMI
percentage_rand = totPercentage*(1-mixinig_factor_PMI)
pointsToretain = int((len(var1)*percentageToStroe)/100.00)
print 'Percentage needed: ' + str(totPercentage)
print 'Points to be taken PMI based: ' + str(pointsToretain)
print 'Points to be taken randomly: ' + str(int((len(var1)*percentage_rand)/100.00))

outfile = '../output/joint_'+samp_type+'_sampled_'+data_set+'_'+arrayName1+'_'+arrayName2+'_'+str(totPercentage)+'.vtp'
pmivol_file = '../output/pmi_vol_p_vel.vti'

Percentage needed: 1
Points to be taken PMI based: 31250
Points to be taken randomly: 0


In [6]:
class Histogram_1D:
    def __init__(self, pmi, bin_id, freq, imp):
        self.pmi = pmi
        self.bin_id = bin_id
        self.freq = freq
        self.imp = imp
    def __repr__(self):
        return repr((self.pmi, self.bin_id, self.freq, self.imp))
    
def compute_3d_to_1d_map (pts, dims):
    return pts[0] + dims[0]*(pts[1]+dims[1]*pts[2])

def compute_2d_to_1d_map (pts, dims):
    return pts[0] + dims[0]*(pts[1])

def compute_1d_to_2d_map (idx, dim):
    a = idx/dim
    b = idx%dim
    return a, b

## compute histogram and PMI
numBins=128
numSamples = np.shape(var1)[0]

hist1 = np.histogram(var1,bins=numBins)
hist2 = np.histogram(var2,bins=numBins)
hist2D = np.histogram2d(var1,var2,bins=numBins)

Array1 = hist1[0]
Array2 = hist2[0]
ArrayComb = hist2D[0]

## compute PMI
PMI = compute_pointwise_mutual_information(Array1,Array2,ArrayComb,numSamples,numBins)

PMI = abs(PMI)

max_pmi = np.max(PMI)
min_pmi = np.min(PMI)
print 'min pmi: ' + str(min_pmi)
print 'max pmi: ' + str(max_pmi)

# normalize PMI
PMI = (PMI-min_pmi)/(max_pmi-min_pmi)
max_pmi = np.max(PMI)
min_pmi = np.min(PMI)

# PMI_flat = np.zeros(numBins*numBins)
# idx=0
# for i in range(numBins):
#         for j in range(numBins):
#             PMI_flat[idx] = PMI[i][j]
#             idx=idx+1
            
PMI_flat = PMI.flatten()

sorted_pmi_arr=[]    
for i in range(numBins*numBins):
    bin_id1,bin_id2 = compute_1d_to_2d_map(i,numBins)
    freq_val = ArrayComb[bin_id1][bin_id2]
    sorted_pmi_arr.append(Histogram_1D(PMI_flat[i],i,freq_val,0.0))
    
    
sorted_pmi_arr = sorted(sorted_pmi_arr, key=attrgetter('pmi'),reverse=True)

min pmi: 0.0
max pmi: 11.96071491498369


In [7]:
# ## Compute pmi based importance function
# print 'points to select: ' + str(pointsToretain)

# pointsToretain_temp = pointsToretain
# pointsPerBin = int(pointsToretain_temp/float(numBins*numBins))
# print 'Initial points per bin to be taken: ' + str(pointsPerBin)

# currentBinFreq=0
# binCounter=0
# BinsLeft=numBins*numBins

# #Compute the sampling/importance function
# while True:
    
#     if pointsPerBin >= sorted_pmi_arr[binCounter].freq:
#         sorted_pmi_arr[binCounter].imp = 1.0
#         BinsLeft = BinsLeft-1
#         pointsToretain_temp = np.ceil(pointsToretain_temp - sorted_pmi_arr[binCounter].freq)
#         binCounter = binCounter+1
#         pointsPerBin = np.ceil(pointsToretain_temp/BinsLeft)

#         if (pointsToretain_temp <=0):
#             break
    
#     elif pointsPerBin < sorted_pmi_arr[binCounter].freq:
#         sorted_pmi_arr[binCounter].imp = sorted_pmi_arr[binCounter].pmi
#         BinsLeft = BinsLeft-1
#         pointsToretain_temp = np.ceil(pointsToretain_temp - sorted_pmi_arr[binCounter].freq*sorted_pmi_arr[binCounter].pmi)
#         binCounter = binCounter+1
#         pointsPerBin = np.ceil(pointsToretain_temp/BinsLeft)
        
#         if (pointsToretain_temp <=0):
#             break
            
           
# for p in range(binCounter,numBins*numBins):
#     sorted_pmi_arr[p].pmi = 0.0
    

# new_adjusted_pmi = np.zeros((numBins,numBins))
# for i in range(numBins*numBins):
#     idx = sorted_pmi_arr[i].bin_id
#     bin_id1,bin_id2 = compute_1d_to_2d_map(idx,numBins)
#     new_adjusted_pmi[bin_id1][bin_id2] = sorted_pmi_arr[i].pmi
    
    
# expected_samples = 0
# for i in range(numBins):
#     for j in range(numBins):
#         expected_samples = expected_samples + new_adjusted_pmi[i][j]*ArrayComb[i][j]
    
# print 'Sampling will pick in expectation: ' + str(int(expected_samples)) + str(' points')     

In [8]:
## Compute pmi based importance function
print 'points to select: ' + str(pointsToretain)

pointsToretain_temp = pointsToretain
pointsPerBin = int(pointsToretain_temp/float(numBins*numBins))
print 'Initial points per bin to be taken: ' + str(pointsPerBin)

currentBinFreq=0
binCounter=0
BinsLeft=numBins*numBins

pts_comes=0

#Compute the sampling/importance function
while True:
    sorted_pmi_arr[binCounter].imp = 1.0
    pts_comes = pts_comes + sorted_pmi_arr[binCounter].freq
    pointsToretain_temp = int(pointsToretain_temp - sorted_pmi_arr[binCounter].freq)
    BinsLeft = BinsLeft-1
    binCounter = binCounter+1
    pointsPerBin = np.ceil(pointsToretain_temp/BinsLeft)

    if (pointsToretain_temp <=0 or BinsLeft == 0):
        break

print pointsToretain_temp, pts_comes  
        
if pointsToretain_temp > 0:
    pointsPerBin = np.ceil(pointsToretain_temp/BinsLeft)   
    for p in range(binCounter,numBins*numBins):
        sorted_pmi_arr[p].imp = pointsPerBin/float(BinsLeft)
else:
    for p in range(binCounter,numBins*numBins):
        sorted_pmi_arr[p].imp = 0.0
    
new_adjusted_pmi = np.zeros((numBins,numBins))
for i in range(numBins*numBins):
    idx = sorted_pmi_arr[i].bin_id
    bin_id1,bin_id2 = compute_1d_to_2d_map(idx,numBins)
    new_adjusted_pmi[bin_id1][bin_id2] = sorted_pmi_arr[i].imp
    
    
expected_samples = 0
for i in range(numBins):
    for j in range(numBins):
        expected_samples = expected_samples + new_adjusted_pmi[i][j]*ArrayComb[i][j]
    
print 'Sampling will pick in expectation: ' + str(int(expected_samples)) + str(' points')     

points to select: 31250
Initial points per bin to be taken: 1
-15 31265.0
Sampling will pick in expectation: 31265 points


In [9]:
######################################################################
def sample_pmi_based_sorted(PMI,data1,data2,arrayName1,arrayName2):
    
    numBins = len(PMI[0])
    
    numPts = data1.GetPointData().GetArray(arrayName1).GetNumberOfTuples()
    dataArr1 = data1.GetPointData().GetArray(arrayName1)
    dataArr2 = data2.GetPointData().GetArray(arrayName2)
    
    range_vals1 = dataArr1.GetRange() 
    range_vals2 = dataArr2.GetRange()
    
    pdata = vtk.vtkPolyData()
    pts_arr = vtk.vtkPoints()
    sampled_arr = vtk.vtkDoubleArray()
    sampled_arr.SetName(arrayName1)
    sampled_arr1 = vtk.vtkDoubleArray()
    sampled_arr1.SetName(arrayName2)
    
    for i in range(numPts):
        randval = random.uniform(0, 1)
        
        dataval1 = dataArr1.GetTuple1(i)
        bin_id1 = int(((dataval1 - range_vals1[0])/(range_vals1[1]-range_vals1[0]))*(numBins-1))
        
        dataval2 = dataArr2.GetTuple1(i)
        bin_id2 = int(((dataval2 - range_vals2[0])/(range_vals2[1]-range_vals2[0]))*(numBins-1))
        
        if randval <= PMI[bin_id1][bin_id2]:
            pts_arr.InsertNextPoint(data1.GetPoint(i))
            sampled_arr.InsertNextTuple1(dataval1)
            sampled_arr1.InsertNextTuple1(dataval2)
    
    pdata.SetPoints(pts_arr)
    pdata.GetPointData().AddArray(sampled_arr)
    pdata.GetPointData().AddArray(sampled_arr1)
    
    return pdata

####################################################################
def insert_corner_pts(data,arrayName1,data1,arrayName2,sampled_data):
    
    dims = data.GetDimensions()
    varArr = data.GetPointData().GetArray(arrayName1)
    varArr1 = data1.GetPointData().GetArray(arrayName2)
    
    pts = [0,0,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [0,dims[1]-1,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [0,0,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [0,dims[1]-1,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [dims[0]-1,0,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [dims[0]-1,dims[1]-1,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [dims[0]-1,0,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [dims[0]-1,dims[1]-1,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    return sampled_data

######################################################################
def sample_random(data,arrayName1,data1,arrayName2,percentage,sampled_data):
    
    totPts = data.GetNumberOfPoints()
    varArr = data.GetPointData().GetArray(arrayName1)
    varArr1 = data1.GetPointData().GetArray(arrayName2)
    numOfPtsNeeded = int((totPts*percentage)/100.0)
    
    print 'Number of randomly selected points: ' + str(numOfPtsNeeded)
    
    rand_ids = np.random.randint(totPts, size=numOfPtsNeeded)
    
    for i in range(len(rand_ids)):
        pts = data.GetPoint(rand_ids[i])
        var_val = varArr.GetTuple1(rand_ids[i])
        var_val1 = varArr1.GetTuple1(rand_ids[i])
        
        sampled_data.GetPoints().InsertNextPoint(pts)
        sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
        sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
        
    return sampled_data

######################################################################
def write_vtp(filename,pdata):
    pdatawriter = vtk.vtkXMLPolyDataWriter()
    pdatawriter.SetFileName(filename)
    pdatawriter.SetInputData(pdata)
    pdatawriter.Write()

In [10]:
## Sample based on importance function
sampled_data = sample_pmi_based_sorted(new_adjusted_pmi,R1.GetOutput(),R2.GetOutput(),arrayName1,arrayName2)

percentagePickedPMI = 100*(sampled_data.GetNumberOfPoints()/float(len(var1)))
percentage_diff = (totPercentage - percentagePickedPMI)

print 'Percentage_diff: ' + str(percentage_diff)
if percentage_diff > 0:
    sampled_data_mixed = sample_random(R1.GetOutput(),arrayName1,R2.GetOutput(),arrayName2,percentage_diff,sampled_data)
    
else:
    sampled_data_mixed = sample_random(R1.GetOutput(),arrayName1,R2.GetOutput(),arrayName2,percentage_rand,sampled_data)

## add the 8 corner points for linear interpolation to work
sampled_data_final = insert_corner_pts(R1.GetOutput(),arrayName1,R2.GetOutput(),arrayName2,sampled_data_mixed)

Percentage_diff: 0.012992
Number of randomly selected points: 406


In [11]:
print 'Final number of sampled points picked is: '+str(sampled_data_final.GetNumberOfPoints())+' and percentage: '+ \
str(100*(sampled_data_final.GetNumberOfPoints()/float(len(var1))))

## write the data out
print outfile
write_vtp(outfile,sampled_data_final)

Final number of sampled points picked is: 31258 and percentage: 1.000256
../output/joint_pmi_sampled_isabel_Pressure_Velocity_1.vtp
